## In this notebook we will perform inference wheater embeddings vector represents subgraph of normal or  anomalous transaction. using our anomaly detection model     
---
**NOTE**: 

In real life scenarios financial transaction are dynamically evolving graphs. Performing anomaly detection inference on graph embeddings in live Transaction Monitoring Systems will require 1st to update the graph and node representations after new transactions arrive. Recomputing entire graph for every newly arrived transaction will lead to unaxeptable delayes and even monitoring system failure. This problem  will be more sever if large amount of updates happen in a short time window.

Contact us at Logical Clocks and we will help you to setup end to end graph based deep anomaly detection live Transaction Monitoring Systems. 

---

In [1]:
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log
42,application_1612689797954_0040,pyspark,idle,Link,Link


SparkSession available as 'spark'.

In [2]:
import json
import numpy as np
from hops import model
from hops.model import Metric

# Query Model Repository for best anomaly detection model

In [3]:
MODEL_NAME="ganAml"
EVALUATION_METRIC="loss"

In [4]:
best_model = model.get_best_model(MODEL_NAME, EVALUATION_METRIC, Metric.MIN)

In [5]:
print('Model name: ' + best_model['name'])
print('Model version: ' + str(best_model['version']))
print(best_model['metrics'])

Model name: ganAml
Model version: 1
{'loss': '0.04624257981777191'}

# Create Model Serving of Exported Model

In [6]:
from hops import serving

In [7]:
MODEL_NAME

'ganAml'

In [8]:
best_model['version']

1

In [9]:
model_path="/Models/" + best_model['name']
model_path

'/Models/ganAml'

In [10]:
# Create serving
model_path="/Models/" + best_model['name']
model_path
response = serving.create_or_update(artifact_path=model_path, serving_name=MODEL_NAME, serving_type="TENSORFLOW", 
                                 model_version=best_model['version'])

Creating a serving for model ganAml ...
Serving for model ganAml successfully created

In [11]:
# List all available servings in the project
for s in serving.get_all():
    print(s.name)

ganAml

In [12]:
# Get serving status
serving.get_status(MODEL_NAME)

'Stopped'

# Start Model Serving Server

In [13]:
if serving.get_status(MODEL_NAME) == 'Stopped':
    serving.start(MODEL_NAME)

Starting serving with name: ganAml...
Serving with name: ganAml successfully started

In [14]:
import time
while serving.get_status(MODEL_NAME) != "Running":
    time.sleep(5) # Let the serving startup correctly
time.sleep(5)

# Send Prediction Requests to the Served Model using Hopsworks REST API

In [15]:
import hsfs
# Create a connection
connection = hsfs.connection()
# Get the feature store handle for the project's feature store
fs = connection.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

In [16]:
eval_td = fs.get_training_dataset("gan_eval_df", 1)

In [17]:
eval_td.show(5)

+------+--------------------+
|target|           embedding|
+------+--------------------+
|     1|[-0.98300076, -0....|
|     1|[0.8909805, 0.094...|
|     1|[-0.4271028, -0.1...|
|     1|[-0.46720815, -0....|
|     1|[-0.8882408, 0.38...|
+------+--------------------+
only showing top 5 rows

In [18]:
def model_server(model_name, input):
    data = {"signature_name": "serving_default", "inputs": [input]}
    return serving.make_inference_request(model_name, data)['outputs']

In [21]:
scored_df = eval_td.read()\
                   .rdd.map(lambda x: (x.target,model_server(MODEL_NAME, np.array(x.embedding).tolist()))).map(lambda f: (f[0],f[1][0]))\
                   .toDF().toDF("target","score")

In [22]:
scored_df.show()

+------+----------+
|target|     score|
+------+----------+
|     1| 11.549015|
|     1|10.8254032|
|     1|8.62070274|
|     1|10.3630981|
|     1|12.0266867|
|     1|11.3408813|
|     1|10.8329468|
|     1|12.2510614|
|     1|11.0315523|
|     1|9.01837921|
|     1|  10.75354|
|     1|10.3517017|
|     1|12.7071629|
|     1|11.4470119|
|     1|10.8549347|
|     1|7.15237951|
|     1|8.49820328|
|     1|   8.21035|
|     1|14.5921297|
|     1|11.1415367|
+------+----------+
only showing top 20 rows